In [3]:
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

In [ ]:
import os
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import json

iam = boto3.client('iam')
role = iam.get_role(RoleName=os.environ.get('SAGEMAKER_EXECUTION_ROLE'))['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'NousResearch/Llama-2-7b-hf',
	'MAX_INPUT_LENGTH': json.dumps(2048),  # Max length of input text
	'MAX_TOTAL_TOKENS': json.dumps(4096),  # Max length of the generation (including input text)
	'MAX_BATCH_TOTAL_TOKENS': json.dumps(8192), 
	'SM_NUM_GPUS': json.dumps(1)
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.9.3"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=600,
  )


In [7]:
predictor.predict({
	"inputs": "My name is Julien and I like to",
})

[{'generated_text': "My name is Julien and I like tomake things. I'm a designer and a developer. I'm also a photographer,"}]

In [ ]:
predictor.endpoint